In [1]:
from pinmongo import *
from random import randint


s = Pin_scrapy()
username='shiertier'
password='20000418Nuo.'
#pin_cloud = Pin_cloud(username=username, password=password)
pin_local = Pin_local(username=username, password=password, host='localhost', port=27000)

c:\Users\jie\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# 获取用户所有画板id
board_ids = s.username_boards('junjietext')

In [ ]:
# 获取该用户所有图片
for board_id in board_ids:
    pics = s.board_pics(board_id)
    # insert_many函数用于插入到本地local数据库的pics集合中,跳过重复错误
    insert_many(pin_local.pics, pics)

In [ ]:
# 获取该用户所有画板相关图片
for board_id in board_ids:
    # relate=True表示获取该画板相关图片
    pics = s.board_pics(board_id,relate=True)
    for pic in pics:
        try:
            pin_local.pics.update_one({"_id": pic["_id"]}, {"$set": pic}, upsert=True)
        except:
            pass

In [2]:
def clean():
    pin_local.pics.update_many({'link':None},{'$unset':{'link':''}})
    pin_local.pics.update_many({'creator':None},{'$unset':{'creator':''}})
    pin_local.pics.update_many({'creator_id':None},{'$unset':{'creator_id':''}})
    pin_local.pics.update_many({'visual_annotation':[]},{'$unset':{'visual_annotation':''}})

In [18]:
pin_local.pics.update_many({},{'$set':{'tree':1}})

UpdateResult({'n': 21987, 'nModified': 0, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [3]:
def one_t(A):
    match_stage = {'$match': {'status.add': {'$exists': False}}}
    skip_stage = {'$skip': randint(1, 64)}
    limit_stage = {'$limit': 1}
    pic = pin_local.pics.aggregate([match_stage, skip_stage, limit_stage]).next()
    va,repinCount = s.get_pic_add_data(pic['_id'])
    if not va and not repinCount:
        pin_local.pics.update_one({'_id': pic['_id']}, {'$set': {'status.add': False}})
        return
    new_data = {
        'visual_annotation': va,
        'repin': repinCount,
        'status.relate': True,
    }
    pin_local.pics.update_one({'_id':pic['_id']},{"$set":new_data})

In [27]:
pin_local.pics.update_many({'status.relate': False},{'$unset':{'status.relate':''}})

UpdateResult({'n': 404, 'nModified': 404, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [ ]:
from concurrent.futures import ThreadPoolExecutor
from functools import partial
def main():
    with ThreadPoolExecutor(max_workers=64) as executor:
        for _ in range(1280):
             executor.submit(partial(one_t, 2))

main()

#while pin_local.pics.find_one({'status.relate': {'$exists': False}}):
#    main()

In [4]:
def list_add_tree(_list, tree):
    for _dict in _list:
        _dict['tree'] = tree
    return _list

In [5]:
def www(tree_search):
    match_stage = {'$match': {'status.relate': {'$exists': False}, 'tree':tree_search}}
    skip_stage = {'$skip': randint(1, 10000)}
    limit_stage = {'$limit': 1}
    pic = pin_local.pics.aggregate([match_stage, skip_stage, limit_stage]).next()
    pics = s.pics_relate_pics(pic['_id'])
    pics = list_add_tree(pics, tree_search+1)
    try:
        insert_many(pin_local.pics, pics)
        pin_local.pics.update_one({'_id': pic['_id']}, {'$set': {'status.relate': True}})
    except:
        pin_local.pics.update_one({'_id': pic['_id']}, {'$set': {'status.relate': False}})
        return

In [19]:
pin_local.pics.count_documents({'tree':1,'status.relate':{'$exists':True}})

21987

In [11]:
pin_local.pics.update_many({'tree':1},{'$set':{'status.relate':True}})

UpdateResult({'n': 21987, 'nModified': 21987, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [18]:
pin_local.pics.create_index('status.relate')
pin_local.pics.create_index('tree')

'tree_1'

In [ ]:
www(2)

In [ ]:
from concurrent.futures import ThreadPoolExecutor
from functools import partial
def main():
    with ThreadPoolExecutor(max_workers=100) as executor:
        for _ in range(1280000):
             executor.submit(partial(www, 2))
main()

In [3]:
!pip install --upgrade setuptools

  Using cached setuptools-69.5.1-py3-none-any.whl.metadata (6.2 kB)
   ---------------------------------------- 0.0/894.6 kB ? eta -:--:--
   ---------------------------------------- 10.2/894.6 kB ? eta -:--:--
   -- ------------------------------------- 61.4/894.6 kB 1.1 MB/s eta 0:00:01
   ------ --------------------------------- 143.4/894.6 kB 1.4 MB/s eta 0:00:01
   -------------- ------------------------- 317.4/894.6 kB 2.2 MB/s eta 0:00:01
   ------------------------------ --------- 686.1/894.6 kB 3.6 MB/s eta 0:00:01
   ---------------------------------------- 894.6/894.6 kB 4.0 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 60.2.0
    Uninstalling setuptools-60.2.0:
      Successfully uninstalled setuptools-60.2.0


DEPRECATION: torchsde 0.2.5 has a non-standard dependency specifier numpy>=1.19.*; python_version >= "3.7". pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of torchsde or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cx-freeze 6.15.10 requires setuptools<69,>=62.6, but you have setuptools 69.5.1 which is incompatible.


In [6]:
! pip uninstall -y pinmongo

Found existing installation: pinmongo 0.1.0
Uninstalling pinmongo-0.1.0:
  Successfully uninstalled pinmongo-0.1.0


In [7]:
! pip install .

Processing c:\users\jie\videos\pin_scrapy
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for pinmongo: filename=pinmongo-0.1.0-py3-none-any.whl size=6745 sha256=7e3f40530bf90c5f179ad95b535b83414d0d89a9622a447cea4865159d3b3419
  Stored in directory: C:\Users\jie\AppData\Local\Temp\pip-ephem-wheel-cache-eti0ofti\wheels\2d\ee\32\c5207e9d0bb01a6b2b93e298ea6f8f2a478ca4bdd95c35d10c
Successfully built pinmongo


DEPRECATION: torchsde 0.2.5 has a non-standard dependency specifier numpy>=1.19.*; python_version >= "3.7". pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of torchsde or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
